In [1]:
import itertools
from typing import (Any, Dict, FrozenSet, Hashable, ItemsView, Iterator, Iterable,
                    KeysView, List, Mapping, NamedTuple, Optional,
                    Sequence, Tuple, Union, ValuesView, cast)

import numpy as np
import networkx

from cirq import ops, abc, value, extension, linalg, circuits
import cirq

from cirq import Pauli, CliffordGate, PauliInteractionGate, PauliString
from cirq.contrib.paulistring import PauliStringPhasor
from cirq.contrib.paulistring import (
    converted_gate_set, convert_and_separate_circuit, pauli_string_reorder_pred,
    move_non_clifford_into_clifford,
    pauli_string_dag_from_circuit
)
from cirq.contrib.paulistring.pauli_string_optimize import (
    optimized_circuit,
    
    merge_equal_strings,
    remove_negligible_strings,
    extract_clifford_strings,
    move_clifford_strings_into_clifford,
)

In [2]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')

In [8]:
c = cirq.testing.nonoptimal_toffoli_circuit()
c = cirq.google.optimized_for_xmon(c)
c = cirq.google.optimized_for_xmon(c)
c

0: ────────────────────────────────────────────────────────────@────────────────────────────────────────@──────────────────────────────────────────────────────────────────────────────@──────────────────────────@───T──────────────────────────
                                                               │                                        │                                                                              │                          │
1: ───W(0.155)^0.5───@───W(0.155)^-0.5───@───W(0.906)^-0.522───@───W(0.428)^0.25───@───W(0.428)^-0.25───@───W(0.928)^0.5───@───W(0.928)^-0.5───@───W(0.928)^-0.5───@───W(0.178)^-0.5───@──────────W(0.678)^0.25───@───W(0.178)^-0.5───Z^-0.678───
                     │                   │                                         │                                       │                   │                   │
2: ───W(0.469)^0.5───@───W(0.469)^-0.5───@───W(0.469)^0.5──────────────────────────@───W(0.813)^0.5────────────────────────@───W(0.813)^-0.5───@───W(0.813)^0.5────@───W(0.313)^0.25───Z^-0.313──────────────────────────────────────────────────

In [9]:
c_opt = optimized_circuit(c)
c_opt

0: ───────────────────────────────────────@──────────────────────────@────────────────────────────────────────────────────────@───────────────────────@───Z^0.25─────────────────
                                          │                          │                                                        │                       │
1: ───X^0.5────@───X^0.5────@───X^-0.25───@───X^0.75───@───X^-0.75───@───X^-0.5───@───X^0.5────@───X^-0.5───@───W(0.75)^0.5───@────────W(0.25)^0.25───@───W(0.75)^0.5───Z^0.75───
               │            │                          │                          │            │            │
2: ───X^-0.5───@───X^-0.5───@───X^-0.5─────────────────@───X^-0.5─────────────────@───X^-0.5───@───X^-0.5───@───X^-0.25───────Z^-1.0─────────────────────────────────────────────

In [6]:
cirq.testing.assert_allclose_up_to_global_phase(
    c.to_unitary_matrix(),
    c_opt.to_unitary_matrix(),
    atol=1e-7,
)

In [16]:
cirq.Symbol('a') * 1

TypeError: unsupported operand type(s) for *: 'Symbol' and 'int'

In [ ]:
cirq.Symbol.__

In [17]:
cirq.Un

cirq.devices.device.Device

In [18]:
cirq.devices.UnconstrainedDevice

cirq.devices.device.Device

In [ ]:
c_conv = converted_gate_set(c)
c_conv

In [ ]:
ops.SingleQubitMatrixGate

In [ ]:
cl, cr = convert_and_separate_circuit(c)
display(cl)
display(cr)

In [ ]:
cirq.testing.assert_allclose_up_to_global_phase(
    c.to_unitary_matrix(),
    (cl+cr).to_unitary_matrix(),
    atol=1e-7,
)

In [ ]:
string_dag = pauli_string_dag_from_circuit(cl)
len(string_dag)

In [ ]:
d2 = networkx.dag.transitive_reduction(string_dag)

In [ ]:
merge_equal_strings(string_dag)
len(string_dag.nodes)

In [ ]:
remove_negligible_strings(string_dag)
len(string_dag.nodes)

In [ ]:
extracted_cliff_string_dag = extract_clifford_strings(string_dag)
len(string_dag.nodes)

In [ ]:
string_dag.to_circuit()

In [ ]:
cr2 = move_clifford_strings_into_clifford(extracted_cliff_string_dag, cr)
cr2

In [ ]:
c_opt = move_non_clifford_into_clifford(string_dag, cr2)
c_opt

In [ ]:
#cirq.ExpandComposite().optimize_circuit(c_opt)
#cirq.google.MergeRotations().optimize_circuit(c_opt)
c_opt = cirq.google.optimized_for_xmon(c_opt)
c_opt

In [ ]:
cirq.testing.assert_allclose_up_to_global_phase(
    c.to_unitary_matrix(),
    c_opt.to_unitary_matrix(),
    atol=1e-7,
)

In [ ]:
g = networkx.DiGraph([(1,2),(2,3),(1,4)])